### 1. Cloning the Dataset

In [1]:
!rm -r /content/OpenLibrary-Books-Scraper

!git clone https://github.com/AshishJangra27/OpenLibrary-Books-Scraper

rm: cannot remove '/content/OpenLibrary-Books-Scraper': No such file or directory
Cloning into 'OpenLibrary-Books-Scraper'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 93 (delta 1), reused 0 (delta 0), pack-reused 88
Unpacking objects: 100% (93/93), 60.81 MiB | 12.89 MiB/s, done.


### 2. Importing Libraries

In [2]:
import os
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
from bs4 import BeautifulSoup

for i in os.listdir('/content/OpenLibrary-Books-Scraper/data'):
  os.system('gunzip /content/OpenLibrary-Books-Scraper/data/' + i)

!python /content/OpenLibrary-Books-Scraper/app.py

df = pd.read_csv('data.csv').drop_duplicates('title_id')

df['book_stats'] = 'Nan'
df['descriptions'] = 'Nan'
df['reading_stats'] = 'Nan'

100% 53/53 [00:15<00:00,  3.42it/s]


In [3]:
def download(st,en):

  book_stats = []
  descriptions = []
  reading_stats = []

  for ids in tqdm(df['title_id'][st:en]):
    link = 'https://openlibrary.org/' + ids
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')

    try:
      reading_stat = '|'.join([ i.text.replace('\n',' ').strip() for i in soup.find('div', class_ = 'readers-stats__wrapper mobile').find_all('li', class_ = 'reading-log-stat')])
    except:
      reading_stat = np.nan
    try:
      book_stat = '|'.join([ i.text.replace('\n',' ').strip() for i in soup.find('div', class_ = 'edition-omniline').find_all('div', class_ = 'edition-omniline-item')])
    except:
      book_stat = np.nan
    try:
      description = soup.find('div', class_ = 'book-description').text.replace('\n',' ').strip()
    except:
      description = np.nan

    reading_stats.append(reading_stat)
    book_stats.append(book_stat)
    descriptions.append(description)

  df['book_stats'][st:en]    = book_stats
  df['descriptions'][st:en]  = descriptions
  df['reading_stats'][st:en] = reading_stats

  df[st:en].to_csv('data/' + str(st) + '_' + str(en) + '.csv', index = False)

In [ ]:
p1 = multiprocessing.Process(target = download, args = (160000 , 170000))
p2 = multiprocessing.Process(target = download, args = (170000 , 180000))
p3 = multiprocessing.Process(target = download, args = (180000 , 190000))
p4 = multiprocessing.Process(target = download, args = (190000 , 200000))
p5 = multiprocessing.Process(target = download, args = (200000 , 210000))
p6 = multiprocessing.Process(target = download, args = (210000 , 220000))
p7 = multiprocessing.Process(target = download, args = (220000 , 230000))
p8 = multiprocessing.Process(target = download, args = (230000 , 240000))

p1.start()
p2.start()
p3.start()
p4.start()
p5.start()
p6.start()
p7.start()
p8.start()

p1.join()
p2.join()
p3.join()
p4.join()
p5.join()
p6.join()
p7.join()
p8.join()

 22%|██▏       | 2229/10000 [1:47:07<12:32:36,  5.81s/it]

In [8]:
csvs = [csv for csv in os.listdir('data') if '.csv' in csv]

df = pd.DataFrame()

for csv in tqdm(csvs):
    df_ = pd.read_csv('data/' + csv )
    df_['category_'] = csv.split('.csv')[0]
    df = pd.concat((df,df_))

df.to_csv('comb3.csv', index = False)

100%|██████████| 8/8 [00:00<00:00,  8.76it/s]


In [1]:
pd.read_csv('data.csv')

NameError: ignored